In [313]:
from gurobipy import *
import numpy as np
import math as mt

In [314]:
# Poser le nombre des varialbles et des contraintes
nbcont=15
nbvar=30
w=[3,2,1]
n=3
p=6
print(w)

[3, 2, 1]


In [315]:
# Creer une matrice utilite
u=np.zeros(6)
u1= np.array([325,225,210,115,75,50])
for i in range(n):
    u=np.vstack((u1,u))

u=np.delete(u, (n), axis=0)
print(u)


[[325. 225. 210. 115.  75.  50.]
 [325. 225. 210. 115.  75.  50.]
 [325. 225. 210. 115.  75.  50.]]


In [316]:
# Creer le vecteur w'
w1=[]
for i in range(n):
    if i==n-1 :
        w1.append(w[i])
    else :
        w1.append(w[i]-w[i+1])

print(w1)

[1, 1, 1]


In [317]:
# Matrice des contraintes
# 1) generer la matrice des contraintes des x
L=np.array([])
L2=np.zeros((6,1))
b=np.zeros(n*p)

u_prim=-1*u

for i in range (n):
    for j in range(p):
         L=np.append(L,[u_prim[i,j]])
    for k in range(nbvar-(n*p)):
         L=np.append(L,0)

    b=np.vstack((L,b))  
    L=np.array([])
b=np.delete(b, (n), axis=0)

for i in range (n):
    for k in range(p):
         L=np.append(L,0)
    for j in range(p):
         L=np.append(L,[u_prim[i,j]])
    for k in range(p):
         L=np.append(L,0)

    b=np.vstack((b,L)) 
    L=np.array([])

for i in range (n):
    for k in range(nbvar-(n*p)):
         L=np.append(L,0)
    for j in range(p):
         L=np.append(L,[u_prim[i,j]])
    
    b=np.vstack((b,L))  
    L=np.array([])


for i in range(n):
    L1=np.identity(p)
    L2=np.concatenate((L1,L2),axis=1)
L2=np.delete(L2,[-1],1)
b=np.vstack((b,L2))



# 2) generer la matrice des contraintes des bik
L=np.identity(n*n)
L=-1*L
for i in range (p):
     L1=np.zeros(n*n)
     L =np.vstack((L,L1))


# 2) concatener la matrice des contraintes des bik et des xj dans la matrice b

b=np.concatenate((L,b),axis=1)

# 3) generer la matrice des contraintes des rk
L2=np.zeros((1,n))
for i in range(n):
    L1=np.identity(n)
    L2=np.vstack((L1,L2))
L2=np.delete(L2,[-1],0)
L1=np.zeros((p,n))
L2=np.vstack((L2,L1))
  
# 2) concatener la matrice des contraintes rk et b
b=np.concatenate((L2,b),axis=1)
print(b)


[[   1.    0.    0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
  -325. -225. -210. -115.  -75.  -50.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    1.    0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
  -325. -225. -210. -115.  -75.  -50.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.    1.   -0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.   -0.
  -325. -225. -210. -115.  -75.  -50.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.]
 [   1.    0.    0.   -0.   -0.   -0.   -1.   -0.   -0.   -0.   -0.   -0.
     0.    0.    0.    0.    0.    0. -325. -225. -210. -115.  -75.  -50.
     0.    0.    0.    0.    0.    0.]
 [   0.    1.    0.   -0.   -0.   -0.   -0.   -1.   -0.   -0.   -0.   -0.
     0.    0.    0.    0.    0.    0. -325. -225. -210. -115.  -75.  -50.
     0.    0.    0.    0.    0.    0.]
 [   0.    0.    1.   -0.   -0.   -0.   -0.   -0.   -1.   -0.   -

In [318]:
# Second membre
S= [0 for i in range (9)]
for i in range(p):
    S.append(1)

print(S)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [319]:
# Coefficients de la fonction objectif
# generer les Coefficients des r
a=[]
for i in range(1,n+1):
    a.append(i*w1[i-1])
# generer les Coefficients des bik
for i in range(n):
    for j in range(n):
        a.append(-1*w1[j])
# generer les Coefficients des x
for i in range (p*n):
    a.append(0)
print(a)


[1, 2, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [320]:
# Creer les models de la fonction objectif
m = Model("Partie2") 

In [321]:
# declaration des variables de decisions
x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((i+1,(j+1)))))
for i in range (n):
    for j in range(p):
        x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d%d" % ((i+1,(j+1)))))

# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var r1>, <gurobi.Var r2>, <gurobi.Var r3>, <gurobi.Var b11>, <gurobi.Var b12>, <gurobi.Var b13>, <gurobi.Var b21>, <gurobi.Var b22>, <gurobi.Var b23>, <gurobi.Var b31>, <gurobi.Var b32>, <gurobi.Var b33>, <gurobi.Var x11>, <gurobi.Var x12>, <gurobi.Var x13>, <gurobi.Var x14>, <gurobi.Var x15>, <gurobi.Var x16>, <gurobi.Var x21>, <gurobi.Var x22>, <gurobi.Var x23>, <gurobi.Var x24>, <gurobi.Var x25>, <gurobi.Var x26>, <gurobi.Var x31>, <gurobi.Var x32>, <gurobi.Var x33>, <gurobi.Var x34>, <gurobi.Var x35>, <gurobi.Var x36>]


In [322]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(nbvar):
    obj += a[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [323]:

# Definition des contraintes
lignes = range(nbcont)
colonnes = range(nbvar)

for i in lignes:
    m.addConstr(quicksum(b[i][j]*x[j] for j in colonnes) <= S[i], "Contrainte%d" % i)



In [324]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 15 rows, 30 columns and 90 nonzeros
Model fingerprint: 0xb0cb6945
Variable types: 12 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 15 rows, 30 columns, 90 nonzeros
Variable types: 12 continuous, 18 integer (18 binary)

Root relaxation: objective 2.000000e+03, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2000.00000    0    4   -0.00000 2000.00000      -     -    0s
H    0     0             